In [11]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
data = {}
for tp in range(4):
    data[tp+1]  = pd.read_csv(f'datos\counted\d{tp+1}_counted.csv', index_col=0)

In [3]:
data[1].columns

Index(['avisoid', 'empresaid', 'avisofechapublicacion', 'avisovacante',
       'mostrarsueldo', 'avisoexperiencia', 'expiracion', 'dias',
       'avisorepublicacion', 'avisocargo', 'areanombre',
       'actividadempresanombre', 'avisocuerpo', 'disponibilidadnombre',
       'avisoduracioncont', 'avisolugartrabajo', 'gradoescolarnombre',
       'situacionestudios', 'avisorequisitos', 'carreras', 'estado',
       'endpagado_o_gratuito', 'nivelnombre', 'carreras_array',
       'sueldoestimado', '_merge', 'avcu_b1', 'avca_b1', 'avre_b1', 'avcu_b2',
       'avca_b2', 'avre_b2', 'avcu_b3', 'avca_b3', 'avre_b3', 'avcu_b4',
       'avca_b4', 'avre_b4', 'teletrabajo_', 'bucket1', 'bucket2', 'bucket3',
       'bucket4', 'wfh', 'duplicates_avisocuerpo', 'duplicates_avisocargo',
       'similars', 'pseudoindex'],
      dtype='object')

Dates

In [4]:
dates = np.concatenate((data[1]['avisofechapublicacion'].unique(), data[2]['avisofechapublicacion'].unique(), 
                        data[3]['avisofechapublicacion'].unique(), data[4]['avisofechapublicacion'].unique()), axis=0)

In [5]:
duplicates_avisocuerpo = pd.concat([data[1][['avisofechapublicacion', 'duplicates_avisocuerpo']], data[2][['avisofechapublicacion', 'duplicates_avisocuerpo']],
                                    data[3][['avisofechapublicacion', 'duplicates_avisocuerpo']], data[4][['avisofechapublicacion', 'duplicates_avisocuerpo']]], axis=0, ignore_index=True)

In [6]:
duplicates_avisocargo = pd.concat([data[1][['avisofechapublicacion', 'duplicates_avisocargo']], data[2][['avisofechapublicacion', 'duplicates_avisocargo']],
                                    data[3][['avisofechapublicacion', 'duplicates_avisocargo']], data[4][['avisofechapublicacion', 'duplicates_avisocargo']]], axis=0, ignore_index=True)

In [9]:
duplicates_avisocuerpo['avisofechapublicacion'] = pd.to_datetime(duplicates_avisocuerpo['avisofechapublicacion'])
duplicates_avisocargo['avisofechapublicacion'] = pd.to_datetime(duplicates_avisocargo['avisofechapublicacion'])

In [12]:
days_before = 45
days_after = 45

In [13]:
window_before = timedelta(days=days_before)
window_after = timedelta(days=days_after)

In [21]:
duplicates_avisocuerpo['duplicates_avisocuerpo']

592458

In [20]:
# Graph
plt.figure(figsize=(12, 6))
plt.plot(duplicates_avisocuerpo['avisofechapublicacion'], duplicates_avisocuerpo['duplicates_avisocuerpo'], marker='o', linestyle='-', color='b', label='avisocuerpo duplicates')
plt.title('Evolution avisocuerpo duplicates')
plt.xlabel('date')
plt.ylabel('Sum of duplicates')
plt.grid(False)

plt.legend()
plt.show()

OverflowError: Exceeded cell block limit in Agg.  Please set the value of rcParams['agg.path.chunksize'], (currently 0) to be greater than 100 or increase the path simplification threshold(rcParams['path.simplify_threshold'] = 0.111111111111 by default and path.simplify_threshold = 0.111111111111 on the input).

<Figure size 1200x600 with 1 Axes>